In [3]:
pip install statsmodels

  Obtaining dependency information for statsmodels from https://files.pythonhosted.org/packages/4b/e4/f9e96896278308e17dfd4f60a84826c48117674c980234ee38f59ab28a12/statsmodels-0.14.4-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for patsy>=0.5.6 from https://files.pythonhosted.org/packages/87/2b/b50d3d08ea0fc419c183a84210571eba005328efa62b6b98bc28e9ead32a/patsy-1.0.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.9 MB 8.6 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/9.9 MB 8.1 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/9.9 MB 10.3 MB/s eta 0:00:01
   ------- -------------------------------- 1.7/9.9 MB 10.0 MB/s eta 0:00:01
   --------- ------------------------------ 2.3/9.9 MB 11.2 MB/s eta 0:00:01
   ----------- ---------------------------- 2.9/9.9 MB 11.0 MB/s eta 0:00:01
   -------------- ------------------------- 3.5/


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.Collecting openpyxl
  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/c1/8b/5fe2cc11fee489817272089c4203e679c63b570a5aaeb18d852ae3cbba6a/et_xmlfile-2.0.0-py3-none-any.whl.metadata
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)




[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.distributions.empirical_distribution import ECDF
import openpyxl

In [2]:
os.makedirs("./figures", exist_ok=True)

In [3]:
main_df = pd.read_csv("DataCoSupplyChainDataset.csv", encoding="ISO-8859-1")
risk_map_df = pd.read_excel("data_dictionary.xlsx")

In [4]:
risk_map_df.rename(columns=lambda x: x.strip(), inplace=True)
risk_map_df.rename(columns={"Field Name": "Field"}, inplace=True)
main_df.columns = [col.strip() for col in main_df.columns]
risk_map_df['Field'] = risk_map_df['Field'].str.strip()

In [5]:
numeric_cols = main_df.select_dtypes(include=['int64', 'float64']).columns
mapped_numeric = risk_map_df[risk_map_df['Field'].isin(numeric_cols)]

In [6]:
for _, row in mapped_numeric.iterrows():
    col = row['Field']
    risk_type = row['Risk Variable Type']
    
    data = main_df[col].dropna()
    unique_vals = data.nunique()
    
    print(f"\n=== {col} ({risk_type}) ===")
    
    if data.empty:
        print("Skipping: No valid data points.")
        continue
    
    if unique_vals == 1:
        print(f"Skipping: Only one unique value ({data.iloc[0]}). Not enough variation.")
        continue

    print(data.describe())

    try:
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        sns.histplot(data, kde=True, bins=50)
        plt.title(f"Histogram of {col} ({risk_type})")

        plt.subplot(1, 2, 2)
        ecdf = ECDF(data)
        plt.plot(ecdf.x, ecdf.y, marker='.', linestyle='none')
        plt.title(f"ECDF of {col}")
        plt.xlabel(col)
        plt.ylabel('ECDF')

        plt.tight_layout()

        save_path = f"./figures/{col}_histogram_ecdf.png"
        plt.savefig(save_path)
        plt.close()
        print(f"Plot saved to {save_path}")
        
    except Exception as e:
        print(f"Plotting failed for {col}: {e}")


=== Days for shipping (real) (Lead Time) ===
count    180519.000000
mean          3.497654
std           1.623722
min           0.000000
25%           2.000000
50%           3.000000
75%           5.000000
max           6.000000
Name: Days for shipping (real), dtype: float64
Plot saved to ./figures/Days for shipping (real)_histogram_ecdf.png

=== Days for shipment (scheduled) (Lead Time) ===
count    180519.000000
mean          2.931847
std           1.374449
min           0.000000
25%           2.000000
50%           4.000000
75%           4.000000
max           4.000000
Name: Days for shipment (scheduled), dtype: float64
Plot saved to ./figures/Days for shipment (scheduled)_histogram_ecdf.png

=== Benefit per order (Other) ===
count    180519.000000
mean         21.974989
std         104.433526
min       -4274.979980
25%           7.000000
50%          31.520000
75%          64.800003
max         911.799988
Name: Benefit per order, dtype: float64
Plot saved to ./figures/Benefit per 

### Explanation

- Central Tendency
    - Look for the main bulk of data (peak in histogram).
    - The ECDF shows where most points accumulate (steepest slope).
- Spread (Variance)
    - Wider histograms mean more variability.
    - ECDF transitions over a wider x-range.
- Skewness
    - Histogram tail longer on one side indicates skew.
    - ECDF curve will stretch further on that side.
- Heavy Tails
    - Histogram has outlier bars far from center.
    - ECDF is flatter in tails before steep rise.
- Multimodality
    - Multiple peaks in histogram suggest subgroups.
    - ECDF will have steps that flatten and then steepen repeatedly.
- Discrete / Categorical-like Numeric Variables
    - Histogram shows clear separated bars.
    - ECDF appears as a step function with jumps.
- Uniform / Identification Variables
    - Histogram bars are roughly equal height.
    - ECDF is a straight diagonal line.
- Outliers
    - Histogram bars far from the main peak.
    - ECDF curve will have early or late jumps.